In [12]:
import json
import os
import subprocess
import tqdm
import logging
import ast
import time
from io import StringIO
import pandas as pd
import numpy as np
import os

logging.getLogger().setLevel(logging.INFO)

out_base_dir = '/media/harryzhang/VolumeWD/DataDump_MRN'
out_query_dir = '/media/harryzhang/VolumeWD/QueryUID'

log_outdir = os.path.join(out_base_dir, 'logs')

if not os.path.isdir(log_outdir):
    os.makedirs(log_outdir)

num_iterations = 1

dcm_file = pd.read_csv('/home/harryzhang/PACS_QUERY/Johnny_Accessions.csv', dtype = str)
dcm_array = np.array(dcm_file)

while num_iterations < 6:
#     fin = open(os.path.join(out_base_dir, 'suid_list.json'), 'r')
#     suid_list = json.load(fin)
#     fin.close()
    
        
    completed_patients_fname = os.path.join(out_base_dir, 'completed_patients.txt')
    try:
        completed_patients_in = open(completed_patients_fname, 'r')
    except:
        completed_patients_in = open(completed_patients_fname, 'w')
        completed_patients_in = open(completed_patients_fname, 'r')
    
    
    completed_list = []
    for line in completed_patients_in:
        completed_list.append(ast.literal_eval(line))
    completed_patients_in.close()

#     new_list = []
#     for line in suid_list:
#         if line not in completed_list:
#             new_list.append(line)
    new_list = []
    for pt in np.arange(0,dcm_array.shape[0]):
        if dcm_array[pt][0] not in completed_list:
            new_list.append(dcm_array[pt])

    if len(new_list) == 0:
        break

    completed_patients_out = open(completed_patients_fname, 'a')

    logging.info("Iter %d, Starting DICOM dump into directory %s" % (num_iterations, out_base_dir,))
    logging.info("Error logs located at %s" % (log_outdir,))
    logging.info("%d studies remain" % (len(new_list),))

#     for suid_pair in tqdm.tqdm(new_list):
#         study_instance_uid = suid_pair[0]
#         patient_id = suid_pair[2]
    for pt_row in new_list:
        print(pt_row)
        patient_outdir = os.path.join(out_base_dir, str(pt_row[0]))
        if not os.path.isdir(patient_outdir):
            os.makedirs(patient_outdir)

        log_out_filename = os.path.join(log_outdir, '%s.%d.log' % (str(pt_row[0]), num_iterations))
        
        fout = open(log_out_filename, 'w')

        rv = subprocess.call('sh %s findscu -b bofa-420-aberle@10.9.94.219:12112 \
        -c WWPACSQR@10.7.1.64:4100 -m AccessionNumber=7622912 -r StudyInstanceUID \
        --out-dir /media/harryzhang/VolumeWD/QueryUID/AccessionNumber-7622912/' % \
                             (DCH4CHE_script, pt_row[2], patient_outdir),stdout=fout , shell=True)
        print('Waiting...')
        time.sleep(30)

        fout.close()
        empty_dirs = [dirpath for (dirpath,dirnames,filenames) in os.walk(patient_outdir) if len(filenames) == 0 and len(dirnames) == 0]
        if len(empty_dirs) == 0:
            completed_patients_out.write('%s\n' % (pt_row[0],))
#             os.remove(log_out_filename)
        else:
            for ed in empty_dirs:
                logging.error("Failure rv (%d) for patient %s, %s modality" % (rv, pt_row[0],ed))
                print("Failure rv (%d) for patient %s, %s modality" % (rv, pt_row[0],ed))
                os.rmdir(ed)
    completed_patients_out.close()

    num_iterations += 1
logging.info("Dump complete")

INFO:root:Iter 1, Starting DICOM dump into directory /media/harryzhang/VolumeWD/Data_Dump_MRN
INFO:root:Error logs located at /media/harryzhang/VolumeWD/Data_Dump_MRN/logs
INFO:root:181 studies remain


['13' '3765588' '7622912' '20120417']
Waiting...


KeyboardInterrupt: 

In [13]:
print('sh %s -L bofa-420-aberle@10.9.94.219:12112 WWPACSQR@10.7.1.64:4100 \
-cmove bofa-420-aberle -qAccessionNumber=%s -cstoredest %s -cstore SC:JPLL' % \
                             (DCH4CHE_script, pt_row[2], patient_outdir))

sh /home/harryzhang/PACS_QUERY/dcm4che-5.17.1-bin/dcm4che-2.0.29/bin/dcmqr -L bofa-420-aberle@10.9.94.219:12112 WWPACSQR@10.7.1.64:4100 -cmove bofa-420-aberle -qAccessionNumber=7622912 -cstoredest /media/harryzhang/VolumeWD/Data_Dump_MRN/13 -cstore SC:JPLL


In [ ]:
/home/harryzhang/PACS_QUERY/dcm4che-5.17.1-bin/dcm4che-5.17.1/bin/movescu -b bofa-420-aberle@10.9.94.219:12112 -c WWPACSQR@10.7.1.64:4100 -cmove bofa-420-aberle -qAccessionNumber=7622912 -cstoredest /media/harryzhang/VolumeWD/Data_Dump_MRN/13 -cstore SC:JPLL


In [1]:
####################################################################################################################
############################# Deidentify DICOM #####################################################################
####################################################################################################################
####################################################################################################################
import deid
import deid.dicom
import deid.config
import os
from deid.utils import get_installdir

In [2]:
from collections import OrderedDict

deid_recipe = OrderedDict([('format', 'dicom'),
             ('header',
              [{'action': 'ADD',
                'field': 'PatientIdentityRemoved',
                'value': 'Yes'},
               {'action': 'REMOVE', 'field': 'endswith:Time'},
               {'action': 'REMOVE', 'field': 'endswith:Date'},
               {'action': 'REMOVE', 'field': 'endswith:time'},
               {'action': 'REMOVE', 'field': 'AdditionalPatientHistory'},
#                {'action': 'REPLACE',
#                 'field': 'PatientID',
#                 'value': 'var:patient_id'},
               {'action': 'REMOVE', 'field': 'PatientID' },
               {'action': 'REMOVE', 'field': 'startswith:IssueDate'},
               {'action': 'REMOVE', 'field': 'IssuerOfPatientID'},
               {'action': 'REMOVE', 'field': 'FillerOrderNumberImagingServiceRequest'},
               {'action': 'REMOVE', 'field': 'OtherPatientIDs'},
               {'action': 'REMOVE', 'field': 'OtherPatientNames'},
               {'action': 'REMOVE', 'field': 'OtherPatientIDsSequence'},
               {'action': 'REMOVE', 'field': 'PatientAddress'},
               {'action': 'REMOVE', 'field': 'PatientBirthName'},
               {'action': 'REMOVE', 'field': 'PatientMotherBirthName'},
               {'action': 'REMOVE', 'field': 'PatientName'},
               {'action': 'REMOVE', 'field': 'PatientsName'},
               {'action': 'REMOVE', 'field': 'ReasonForStudy'},
#                {'action': 'REMOVE', 'field': 'contains:Trial'},
               {'action': 'REMOVE', 'field': 'startswith:PatientTelephoneNumber'},
               {'action': 'REMOVE', 'field': 'ReferringPhysicianName'},
               {'action': 'REMOVE', 'field': 'ConsultingPhysicianName'},
               {'action': 'REMOVE', 'field': 'EvaluatorName'},
               {'action': 'REMOVE', 'field': 'PerformedStationName'},
               {'action': 'REMOVE', 'field': 'SecondaryReviewerName'},
               {'action': 'REMOVE', 'field': 'PersonName'},
               {'action': 'REMOVE', 'field': 'PersonAddress'},
               {'action': 'REMOVE', 'field': 'ReferringPhysicianAddress'},
               {'action': 'REMOVE', 'field': 'ReferringPhysicianTelephoneNumbers'},
               {'action': 'REMOVE', 'field': 'ReferringPhysicianIdentificationSequence'},
               {'action': 'REMOVE', 'field': 'ConsultingPhysicianIdentificationSequence'},
               {'action': 'REMOVE', 'field': 'PhysicianApprovingInterpretation'},
               {'action': 'REMOVE', 'field': 'PhysiciansOfRecord'},
               {'action': 'REMOVE', 'field': 'PhysiciansOfRecordIdentificationSequence'},
               {'action': 'REMOVE', 'field': 'PerformingPhysicianName'},
               {'action': 'REMOVE', 'field': 'PerformingPhysicianIdentificationSequence'},
               {'action': 'REMOVE', 'field': 'PhysiciansReadingStudyIdentificationSequence'},
               {'action': 'REMOVE', 'field': 'RequestingPhysician'},
               {'action': 'REMOVE', 'field': 'ScheduledPerformingPhysicianIdentificationSequence'},
               {'action': 'REMOVE', 'field': 'RequestingPhysicianIdentificationSequence'},
               {'action': 'REMOVE', 'field': 'HumanPerformerName'},
               {'action': 'REMOVE', 'field': 'ContactDisplayName'},
               {'action': 'REMOVE', 'field': 'PerformingPhysicianName'},
               {'action': 'REMOVE', 'field': 'NameOfPhysiciansReadingStudy'},
               {'action': 'REMOVE', 'field': 'OperatorsName'},
               {'action': 'REMOVE', 'field': 'ScheduledPerformingPhysicianName'},
               {'action': 'REMOVE', 'field': 'ReviewerName'},
               {'action': 'REMOVE', 'field': 'NamesOfIntendedRecipientsOfResults'},
               {'action': 'REMOVE', 'field': 'SourceApplicatorName'},
               {'action': 'REMOVE', 'field': 'ClinicalTrialSponsorName'},
               {'action': 'REMOVE', 'field': 'ContentCreatorName'},
               {'action': 'REMOVE', 'field': 'ClinicalTrialProtocolEthicsCommitteeName'},
               {'action': 'REMOVE', 'field': 'RegionOfResidence'},
               {'action': 'REMOVE', 'field': 'CurrentPatientLocation'},
               {'action': 'REMOVE', 'field': 'PatientComments'},
               {'action': 'REMOVE', 'field': 'PatientTransportArrangements'},
               {'action': 'REMOVE', 'field': 'PatientDeathDateInAlternativeCalendar'},
               {'action': 'REMOVE', 'field': 'PatientInstitutionResidence'},
               {'action': 'REMOVE', 'field': 'PerformedLocation'},
               {'action': 'REMOVE', 'field': 'ScheduledStudyLocation'},
               {'action': 'REMOVE', 'field': 'ScheduledStudyLocationAETitle'},
               {'action': 'REMOVE', 'field': 'OrderEntererLocation'},
               {'action': 'REMOVE', 'field': 'AssignedLocation'},
               {'action': 'REMOVE', 'field': 'AccessionNumber'},
               {'action': 'REMOVE', 'field': 'InstitutionAddress'},
               {'action': 'REMOVE', 'field': 'InstitutionalDepartmentName'},
               {'action': 'REMOVE', 'field': 'InstitutionCodeSequence'},
               {'action': 'REMOVE', 'field': 'InstitutionName'},
               {'action': 'REMOVE', 'field': 'StationName'},
#                {'action': 'REMOVE', 'field': 'SeriesInstanceUID'},
               {'action': 'REMOVE', 'field': 'StudyInstanceUID'},
               {'action': 'REMOVE', 'field': 'SOPInstanceUID'},
               
              ])])

In [3]:
from deid.dicom import get_files, replace_identifiers
from deid.utils import get_installdir
from deid.data import get_dataset
import os
from deid.config import DeidRecipe
from deid.dicom import get_identifiers

import pandas as pd
import numpy as np

dcm_file = pd.read_csv('2018_montage_dcm_test.csv', dtype = str)
dcm_array = np.array(dcm_file)

out_base_dir = '/home/jennifer/Desktop/Data_Dump_MRN'
clean_dir = '/home/jennifer/Desktop/Dump_Clean/'

for row in dcm_array:
    patient_outdir = os.path.join(out_base_dir, str(row[0]))
    output_dir = os.path.join(clean_dir, str(row[0]))
    if not os.path.isdir(output_dir):
            os.makedirs(output_dir)
    
    dicom_files = list(get_files(patient_outdir))
    
    items = get_identifiers(dicom_files)

    recipe = DeidRecipe(deid_recipe)

    cleaned_files = replace_identifiers(dicom_files=dicom_files,
                                        deid=recipe,
                                        ids=items,
                                        output_folder=output_dir)

WARNING Cannot read input file /home/jennifer/Desktop/Data_Dump_MRN/190078, skipping.


In [5]:
####################################################################################################################
############################# Move DICOM (if study level is better than series level) ##############################
####################################################################################################################
####################################################################################################################
import pydicom
from pydicom.data import get_testdata_files
from pydicom.tag import Tag
import os

import pandas as pd
import numpy as np

dcm_file = pd.read_csv('2018_montage_dcm_test.csv', dtype = str)
dcm_array = np.array(dcm_file)


for row in dcm_array:
    patient_outdir = os.path.join(clean_dir, row[0])
#     print(patient_outdir)
    #iterate through the files
    for root, dirs, files in os.walk(patient_outdir):
        for file in files:
            dicom_path = os.path.join(root, file)
#             print(dicom_path)
            #read dicom file
            ds = pydicom.dcmread(dicom_path)
            #find modality
            modality = (ds[Tag(0x008103e)].value)
            mod_dir = os.path.join(patient_outdir, modality)
#             print(modality)
            #create directory if it doesnt exist
            if not os.path.isdir(mod_dir):
                os.makedirs(mod_dir)
            #name the new directory using the modality
            new_dir = os.path.join(patient_outdir, modality,file)
#             print(new_dir)
#             #this combines - will delete eventually
#             new_dir = os.path.join(patient_outdir,file)

            #move the file
            os.rename(dicom_path, new_dir)